## Test Data Prediction
### Feat7
This notebook will be used to open test.csv file and use designated model to predict the test data for kaggle submission.

In [1]:
%%bash
git chechout 'feat6/wip'
git fetch

  CV_check
  EDA1/wip
  cost1/feat1/wip
  cost1/feat2/wip
  cost1/wip
  feat1/score1/wip
  feat1/wip
  feat2/score1/wip
  feat2/wip
  feat3/wip
  feat4/wip
  feat5/wip
  feat6/wip
* feat7(EDA1)/wip
  master


In [2]:
%%bash
git add 'Test_data.ipynb'
git commit -m 'Prepare code for predicting for submission file'
git push --set-upstream origin master

[feat7(EDA1)/wip 8a2194b] Branched to feat7
 1 file changed, 8 insertions(+), 4 deletions(-)
Branch master set up to track remote branch master from origin.


Everything up-to-date


In [30]:
model_ver = 'feat7.sav'

In [31]:
%matplotlib inline 
import requests 
import StringIO
import zipfile
import numpy as np
import pandas as pd # pandas
import matplotlib.pyplot as plt # module for plotting 

# If this module is not already installed, you may need to install it. 
# You can do this by typing 'pip install seaborn' in the command line
import seaborn as sns 

import sklearn.model_selection
import pickle

Load data from csv file then prepare the data before inputting into the trained model.

In [32]:
test_ex = pd.read_csv('gender_submission.csv')
test_ex.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [39]:
X = train.copy()
#X = X.drop('Survived',axis=1)
X_plot = train.copy()
X = X.drop('PassengerId',axis=1)  # Shouldn't relate to survival chance
X = X.drop('Cabin',axis=1)
X = X.drop('Ticket',axis=1)

#Mirror action on X to test
#test = test.drop('PassengerID',axis=1)
test = test.drop('PassengerId',axis=1)
test = test.drop('Cabin',axis=1)
test = test.drop('Ticket',axis=1)

#X = X.drop('Name',axis=1)
Y = pd.DataFrame(train['Survived'])

X_plot['Initial']=0
for i in X_plot:
    X_plot['Initial']=X_plot.Name.str.extract('([A-Za-z]+)\.') #lets extract the Salutations
#X_plot['Initial'].value_counts()

#Change data in X also (since its the real training data)
X['Initial']=0
for i in X:
    X['Initial']=X.Name.str.extract('([A-Za-z]+)\.') #lets extract the Salutations
#Also perform same actions on test dataframe
test['Initial']=0
for i in X:
    test['Initial']=test.Name.str.extract('([A-Za-z]+)\.') #lets extract the Salutations
X['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],
                        ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr'],inplace=True)
test['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],
                        ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr'],inplace=True)

X['Age'][(X['Initial']=='Mr') & (X['Age'].isnull())] = X[X['Initial']=='Mr']['Age'].mean()
X['Age'][(X['Initial']=='Miss') & (X['Age'].isnull())] = X[X['Initial']=='Miss']['Age'].mean()
X['Age'][(X['Initial']=='Mrs') & (X['Age'].isnull())] = X[X['Initial']=='Mrs']['Age'].mean()
X['Age'][(X['Initial']=='Master') & (X['Age'].isnull())] = X[X['Initial']=='Master']['Age'].mean()
X['Age'][(X['Initial']=='other') & (X['Age'].isnull())] = X[X['Initial']=='other']['Age'].mean()

test['Age'][(test['Initial']=='Mr') & (test['Age'].isnull())] = test[test['Initial']=='Mr']['Age'].mean()
test['Age'][(test['Initial']=='Miss') & (test['Age'].isnull())] = test[test['Initial']=='Miss']['Age'].mean()
test['Age'][(test['Initial']=='Mrs') & (test['Age'].isnull())] = test[test['Initial']=='Mrs']['Age'].mean()
test['Age'][(test['Initial']=='Master') & (test['Age'].isnull())] = test[test['Initial']=='Master']['Age'].mean()
test['Age'][(test['Initial']=='other') & (test['Age'].isnull())] = test[test['Initial']=='other']['Age'].mean()

num_attri = ['Age','SibSp','Parch','Fare']
cat_attri = [column for column in X if (column not in num_attri and column != 'Survived')]

for column in (cat_attri):
    #Lets just keep every categorical attributes for plotting
    tempDF = pd.get_dummies(X[column],prefix=column)
    X.drop(column,axis=1,inplace=True)
    X = X.join(tempDF)
for column in (cat_attri): #Create dummy for test
    #Lets just keep every categorical attributes for plotting
    tempDF = pd.get_dummies(test[column],prefix=column)
    test.drop(column,axis=1,inplace=True)
    test = test.join(tempDF)

#Concat X(train data) with X from test data to avoid mismatch feature
train_objs_num = len(X)
dataset = pd.concat(objs=[X, test], axis=0)
#dataset = pd.get_dummies(dataset)
X = dataset[:train_objs_num].copy()
test = dataset[train_objs_num:].copy()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,34.5,NaN,Q,7.8292,"Kelly, Mr. James",0,892,3,male,0,NaN,330911
1,47.0,NaN,S,7.0000,"Wilkes, Mrs. James (Ellen Needs)",0,893,3,female,1,NaN,363272
2,62.0,NaN,Q,9.6875,"Myles, Mr. Thomas Francis",0,894,2,male,0,NaN,240276
3,27.0,NaN,S,8.6625,"Wirz, Mr. Albert",0,895,3,male,0,NaN,315154
4,22.0,NaN,S,12.2875,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,896,3,female,1,NaN,3101298


In [41]:
X_test = test.copy()

Age                           True
Fare                          True
Parch                        False
SibSp                        False
Survived                      True
Cabin_A11                    False
Cabin_A18                    False
Cabin_A21                    False
Cabin_A29                    False
Cabin_A34                    False
Cabin_A9                     False
Cabin_B10                    False
Cabin_B11                    False
Cabin_B24                    False
Cabin_B26                    False
Cabin_B36                    False
Cabin_B41                    False
Cabin_B45                    False
Cabin_B51 B53 B55            False
Cabin_B52 B54 B56            False
Cabin_B57 B59 B63 B66        False
Cabin_B58 B60                False
Cabin_B61                    False
Cabin_B69                    False
Cabin_B71                    False
Cabin_B78                    False
Cabin_C101                   False
Cabin_C105                   False
Cabin_C106          

In [28]:
# load the model from disk
loaded_model = pickle.load(open(model_ver, 'rb'))
accuracy_score(y_test,clf2.predict(X_test))
result = loaded_model.score(X_test, Y_test)
print(result)

NameError: name 'y_test' is not defined